This is a script attempting to optimize the return of investment (ROI) on the equality portfolio.

In [1]:
# Import python libraries
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import yfinance as yf
from pypfopt import EfficientFrontier, risk_models, expected_returns
plt.style.use("fivethirtyeight")


# User's input total investment amount, number of equilities and equility symbols

# Total investment amount

total_investment = int(input("Please Input the Total of Investment: "))
n_equity = int(input("Please Input the Number of Equities: "))
equities = [str(input("Please Input the Symbol of Equities: ")).upper() for i in range(n_equity)]

print("\nCurrent Portfolio Amount: " + "$" + str(total_investment))
print("The Number of Equities: " + str(n_equity) + '\n')
print("Current Equity: ") 
[print(equity) for equity in equities]
print('\n')

# obtaining historical price

# choosing date to retrieve historical price
startDate = "2016-01-01"
endDate = datetime.today().strftime("%Y-%m-%d")

df = pd.DataFrame()
                                    
for equity in equities:
    # retrieving historical price from yahoo finance and return the adje                               
    df[equity] = yf.download(equity, startDate, endDate)['Adj Close']
    
fig, ax = plt.subplots(figsize=(12, 12))

ax = plt.gca()
title = 'Adj. Close Price'

for sym in list(df.columns):
    plt.plot(df[sym], label = sym)


ax.set(xlabel="Date",
       ylabel="Adj. Price USD $",
       title = title)
plt.legend(list(df.columns), loc = 'upper left')
plt.show()

# Portfolio Optimization for Each Assest

# Expected returns and annualized sample covariance matrix

mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

# Optimizing for max sharpe ratio based on volatility 
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

print("\nOptimazation Parameters and Shares Allocation\n")
print(cleaned_weights)
ef.portfolio_performance(verbose = True)

# Allocating numbers of shares for each assest

from pypfopt import DiscreteAllocation, get_latest_prices

latest_prices = get_latest_prices(df)
weights = cleaned_weights
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value = total_investment)

allocation, leftover = da.lp_portfolio() # creating variables, allocation and leftover

print('\nShares Allocation:', allocation)
print('Funds Remaining: ${:.2f}'.format(leftover))





Please Input the Total of Investment: 275000
Please Input the Number of Equities: 4
Please Input the Symbol of Equities: aapl
Please Input the Symbol of Equities: tsla
Please Input the Symbol of Equities: goog
Please Input the Symbol of Equities: nflx

Current Portfolio Amount: $275000
The Number of Equities: 4

Current Equity: 
AAPL
TSLA
GOOG
NFLX


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



Optimazation Parameters and Shares Allocation

OrderedDict([('AAPL', 0.54674), ('TSLA', 0.17681), ('GOOG', 0.0), ('NFLX', 0.27645)])
Expected annual return: 36.1%
Annual volatility: 28.4%
Sharpe Ratio: 1.20

Shares Allocation: {'AAPL': 453.0, 'TSLA': 55.0, 'NFLX': 181.0}
Funds Remaining: $171.60
